In [1]:
import requests
import pandas as pd
from datetime import datetime as dt
from xml.etree import ElementTree as ET
from config import userid, password
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='

tracking_df = pd.read_csv('TrackingList.csv',dtype={'Client Nos.':str})

In [3]:
# Build a list of delivered packages based on API calls and save it to a file:
delivered_list = []
rn = dt.now()
filename = 'Output\deliveredlist' + rn.strftime('%Y%m%d_%H%M') + '.txt'
deliveredfile = open(filename, 'a')
deliveredfile.write(
    'Packages delivered as of ' + rn.strftime('%d %B %Y, %H:%M') + ':\n'
)
for num in tracking_df['Tracking Nos.']:
    response = requests.get(base_url + \
                        f'<TrackRequest USERID="{userid}" PASSWORD="{password}">\
                            <TrackID ID="{num}"></TrackID>\
                                </TrackRequest>')
    soup = BeautifulSoup(response.content, 'xml')
    # Three different phrases might indicate that the package has reached the recipient:
    condition1 = 'Your item was delivered' in str(response.content)
    condition2 = 'Your item has been delivered' in str(response.content)
    condition3 = 'Your item was picked up' in str(response.content)

    if condition1 | condition2 | condition3:
        delivered_list.append(num)
        spots = tracking_df[tracking_df['Tracking Nos.'] == num].index
        # In most cases, there will only be one spot in spots.
        for spot in spots:
            deliveredfile.write(f'\n{tracking_df.iloc[spot,0]}, {tracking_df.iloc[spot,1]}, {tracking_df.iloc[spot,2]}')
            deliveredfile.write('\n\n' + soup.TrackSummary.text + '\n')

In [4]:
# Print a list of the delivered packages to the terminal:
if len(delivered_list) == 0:
    print('No new packages have been delivered.')
    deliveredfile.write('\nNo packages delivered since last report.')
else:
    print('The following ' + str(len(delivered_list)) + ' packages have been delivered:')
    [print(f'{id}') for id in delivered_list]

deliveredfile.close()

The following 7 packages have been delivered:
9114902200852144849218
9114902200852144849324
9114902200852144849294
9589071052701128330816
9589071052701128330960
9589071052701128330960
9589071052701128330960


In [7]:
print(delivered_list)
tracking_df.tail(20)

['9114902200852144849218', '9114902200852144849324', '9114902200852144849294', '9589071052701128330816', '9589071052701128330960', '9589071052701128330960', '9589071052701128330960']


,Tracking Nos.,Client Nos.,Comments
102,9589071052701128330908,80901.02,Response to Letter CP162A
103,9589071052701128330625,63769.02,Form 709
104,70211970000045303011,4522.2,Notice CP161
105,9589071052701128330472,5844.02,LA-ES June Payment
106,9589071052701128330489,5842.02,LA-ES June Payment
107,9589071052701128330182,12710.02,1040ES Voucher 2
108,9114902200852144849157,55583.02,Source Docs
109,9114902200852144849218,93108.02,Form 709
110,9114902200852144849201,46961.02,Source Docs
111,9114902200852144849188,63768.02,Source Docs


In [9]:
for id in delivered_list:
    spots = tracking_df[tracking_df['Tracking Nos.'] == id].index
    for spot in spots:
        print(spot)
        # tracking_df.drop(tracking_df[spot], inplace=True)

print(tracking_df.iloc[109])

109
112
113
115
117
118
119
117
118
119
117
118
119
Tracking Nos.    9114902200852144849218
Client Nos.                    93108.02
Comments                       Form 709
Name: 109, dtype: object


In [11]:
# With the file written, ask about removing the packages from the list:
if len(delivered_list) != 0:
    removedelivered = 'x'
    while removedelivered.lower() != 'y' and removedelivered.lower() != 'n':
        removedelivered = input('Would you like to remove them from the tracking list? (y/n) ')
        if removedelivered == 'y':
            for id in delivered_list:
                spots = tracking_df[tracking_df['Tracking Nos.'] == id].index
                for spot in spots:
                    tracking_df.drop(spot, inplace=True) # Fixed. You can just pass the index into .drop and drop the row.
                # In most cases, there will only be one spot in spots.
            tracking_df.to_csv('Output\TrackingListB' + rn.strftime('%Y%B%d_%H%M') + '.csv',index=False)
            tracking_df.to_csv('TrackingListB.csv',index=False)
        elif removedelivered != 'y' and removedelivered != 'n':
            print('Please answer with the letter y for Yes or the letter n for No.')